<a href="https://colab.research.google.com/github/Ilicio/TensorFlow/blob/main/tf2_image_classification_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import explainable_ai_sdk

from explainable_ai_sdk.metadata.tf.v2 import SaveModelMetadataBuilder
from PIL import image

# Model trained on ImageNet

In [ ]:
img_net_dir = 'imagenet_model'
imgnet_builder = SaveModelMetadataBuilder(img_net_dir)

# Generate explanation metadata

Using the Explainable AI SDK, we'll create a reference to our saved model by pointing to the SaveModel directory within our notebooks instance

In [ ]:
imgnet_builder.set_image_metadata('input_1')
imgnet_builder.get_metadata()
imgnet_builder.save_metadata(img_net_dir)

# Preview a test image

In [ ]:
im = Image.open('jellyfish.jpg').convert('RGB')
im = im.resize((224,224))
imarr = np.array(im)
plt.imshow(im)

# Get a prediction and explanation

In [ ]:
lm = explainable_ai_sdk.load_model_from_local_path(
    img_net_dir,
    explainable_ai_sdk.XraiConfig()
)
response = lm.explain([{'input_1': imarr / 255}])
response[0].visualize_attributions()

# Get the label
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
print('Predicted label:', imagenet_labels[108])

In [ ]:
# Show a legend of pixel importance
im = Image.open('key.png')
plt.imshow(im)

# Chest x-ray model

In [ ]:
xray_model_dir = 'chest_xray_model'
builder = SaveModelMetadataBuilder(xray_model_dir)
builder.set_image_metadata('vgg16_input', input_baselines=[[0]])

In [ ]:
builder.save_metadata(xray_model_dir)

In [ ]:
builder.get_metadata()

# Preview a test image

The test image we'll be sending toour model contains Cardiomegaly

In [ ]:
im = Image.open('cardio-test.png').convert('RGB')
im = im.resize((224,224))
imarr = np.array(im)
plt.imshow(im)

# Get a prediction and explanation

In [ ]:
lm = explainable_ai_sdk.load_model_from_local_path(
    xray_model_dir,
    explainable_ai_sdk.XraiConfig(step_count=10)
)
response = lm.explain([{'vgg16_input': imarr / 255}])
response[0].visualize_attributions()

# Deploy the model to AI Platform with explanations

In [ ]:
!gsutil -m cp -r ./chest_xray_model gs://chest_xray_model/

In [ ]:
!gcloud beta ai-platform versions create 'v1' \
--model 'chest_xray' \
--origin 'gs://chest_xray_model' \
--runtime-version 2.1 \
--framework TENSORFLOW \
--python-version 3.7 \
---machine-type n1-standard-4 \
--explanation-method integrated-gradients \
--num-integral-steps 25